This notebook is built on the [MCTS Starter](https://www.kaggle.com/code/yunsuxiaozi/mcts-starter) notebook by @yunsuxiaozi.

Modifications in Version 1:  

1. Added XGB and CB models.
2. LGBM hyperparameters tuned. 
3. Incorporated OOF CV predictions,

Version 2: 
Just added the changes incorporated to Version 1 from the original notebook. You can click on Version 1 and further build on it. 

Version 3: LB 0.445
Tested waters with number of folds, changed the folds from 5 to 10 (CV decreased to 0.419, incorporated weighted ensemble by @swimmy), lb increased to 0.445

Version 5: Removed XGB 

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Libraries</h1></span>

In [ ]:
import polars as pl#和pandas类似,但是处理大型数据集有更好的性能.
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#对矩阵进行科学计算的库
#这里使用groupkfold
from sklearn.model_selection import GroupKFold
#model lightgbm回归模型,日志评估
from  lightgbm import LGBMRegressor,log_evaluation,early_stopping
from sklearn.feature_extraction.text import TfidfVectorizer#将文本数据转换为tfidf特征
import gc#垃圾回收的库
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read Data</h1></span>

In [ ]:
train=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
train=train.to_pandas()
print(f"len(train):{len(train)}")
test=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
test=test.to_pandas()
print(f"len(test):{len(test)}")
test.head()

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Preprocessor</h1></span>

### Feature engineering, model training, and inference are all written in this class.

####  1.Based on the description on the data introduction page, the features of agent1 and agent2 were constructed.

Agent String Descriptions
All agent string descriptions in training and test data are in the following format: MCTS-<SELECTION>-<EXPLORATION_CONST>-<PLAYOUT>-<SCORE_BOUNDS>, where:

<SELECTION> is one of: UCB1, UCB1GRAVE, ProgressiveHistory, UCB1Tuned. These are different strategies that may be used within the Selection phase of the MCTS algorithm.
    
EXPLORATION_CONST is one of: 0.1, 0.6, 1.41421356237. These are three different values that we have tested for the "exploration constant" (a numeric hyperparameter shared among all of the tested Selection strategies).
    
PLAYOUT is one of: Random200, MAST, NST. These are different strategies that may be used within the Play-out phase of the MCTS algorithm.
    
SCORE_BOUNDS is one of: true or false, indicating whether or not a "Score-Bounded" version of MCTS (a version of the algorithm that can prove when certain nodes in its search tree are wins/losses/draws under perfect play, and adapt the search accordingly).
    
For example, an MCTS agent that uses the UCB1GRAVE selection strategy, an exploration constant of 0.1, the NST play-out strategy, and Score Bounds, will be described as MCTS-UCB1GRAVE-0.1-NST-true.
    
    
#### 2.onehot and tfidf is simple.
    
#### 3.Removed some useless features from manual selection and model selection.
    


In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb


In [ ]:
class Preprocessor():
    def __init__(self,seed=1212,target='utility_agent1',train=None,num_folds=5):
        self.seed=seed
        self.target=target
        self.train=train
        self.models=[]#训练和推理的模型
        self.tfidfs=[]#字符串的tfidf模型
        self.num_folds=num_folds
        self.models_lgb = []  # LightGBM models
        #self.models_xgb = []  # XGBoost models
        self.models_cat = []  # CatBoost models

        
    #清理df的字符串的列
    def clean(self,df,col):
        #字符串缺失值填充
        df[col]=df[col].fillna("nan")
        #字符串转换成小写
        df[col]=df[col].apply(lambda x:x.lower())
        #考虑到这种字符串 ‘MCTS-UCB1-0.6-NST-false‘
        ps='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        for p in ps:
            df[col]=df[col].apply(lambda x:x.replace(p,' '))
        return df
        #遍历表格df的所有列修改数据类型减少内存使用
    def reduce_mem_usage(self,df, float16_as32=True):
        #memory_usage()是df每列的内存使用量,sum是对它们求和, B->KB->MB
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

        for col in df.columns:#遍历每列的列名
            col_type = df[col].dtype#列名的type
            if col_type != object and str(col_type)!='category':#不是object也就是说这里处理的是数值类型的变量
                c_min,c_max = df[col].min(),df[col].max() #求出这列的最大值和最小值
                if str(col_type)[:3] == 'int':#如果是int类型的变量,不管是int8,int16,int32还是int64
                    #如果这列的取值范围是在int8的取值范围内,那就对类型进行转换 (-128 到 127)
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    #如果这列的取值范围是在int16的取值范围内,那就对类型进行转换(-32,768 到 32,767)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    #如果这列的取值范围是在int32的取值范围内,那就对类型进行转换(-2,147,483,648到2,147,483,647)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    #如果这列的取值范围是在int64的取值范围内,那就对类型进行转换(-9,223,372,036,854,775,808到9,223,372,036,854,775,807)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:#如果是浮点数类型.
                    #如果数值在float16的取值范围内,如果觉得需要更高精度可以考虑float32
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        if float16_as32:#如果数据需要更高的精度可以选择float32
                            df[col] = df[col].astype(np.float32)
                        else:
                            df[col] = df[col].astype(np.float16)  
                    #如果数值在float32的取值范围内，对它进行类型转换
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    #如果数值在float64的取值范围内，对它进行类型转换
                    else:
                        df[col] = df[col].astype(np.float64)
        #计算一下结束后的内存
        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        #相比一开始的内存减少了百分之多少
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

        return df
        
    def FE(self,df,mode='train'):
        print(f"FE:{mode}")
        
        print("agent1 agent2 feature")
        cols=['selection','exploration_const','playout','score_bounds']
        for i in range(len(cols)):
            for j in range(2):
                df[f'{cols[i]}{j+1}']=df[f'agent{j+1}'].apply(lambda x:x.split('-')[i+1])
        

        print(f"one_hot_encoder")
        #训练集上nunique 大于2小于10的cols
        onehot_cols=[['NumOffDiagonalDirections', [0.0, 4.82, 2.0, 5.18, 3.08, 0.06]], ['NumLayers', [1, 0, 4, 5]], ['NumPhasesBoard', [3, 2, 1, 5, 4]], ['NumContainers', [1, 4, 3, 2]], ['NumDice', [0, 2, 1, 4, 6, 3, 5, 7]], ['ProposeDecisionFrequency', [0.0, 0.05, 0.01]], ['PromotionDecisionFrequency', [0.0, 0.01, 0.03, 0.02, 0.11, 0.05, 0.04]], ['SlideDecisionToFriendFrequency', [0.0, 0.19, 0.06]], ['LeapDecisionToEnemyFrequency', [0.0, 0.04, 0.01, 0.02, 0.07, 0.03, 0.14, 0.08]], ['HopDecisionFriendToFriendFrequency', [0.0, 0.13, 0.09]], ['HopDecisionEnemyToEnemyFrequency', [0.0, 0.01, 0.2, 0.03]], ['HopDecisionFriendToEnemyFrequency', [0.0, 0.01, 0.09, 0.25, 0.02]], ['FromToDecisionFrequency', [0.0, 0.38, 1.0, 0.31, 0.94, 0.67]], ['ProposeEffectFrequency', [0.0, 0.01, 0.03]], ['PushEffectFrequency', [0.0, 0.5, 0.96, 0.25]], ['FlipFrequency', [0.0, 0.87, 1.0, 0.96]], ['SetCountFrequency', [0.0, 0.62, 0.54, 0.02]], ['DirectionCaptureFrequency', [0.0, 0.55, 0.54]], ['EncloseCaptureFrequency', [0.0, 0.08, 0.1, 0.07, 0.12, 0.02, 0.09]], ['InterveneCaptureFrequency', [0.0, 0.01, 0.14, 0.04]], ['SurroundCaptureFrequency', [0.0, 0.01, 0.03, 0.02]], ['NumPlayPhase', [1, 2, 3, 4, 5, 6, 7, 8]], ['LineLossFrequency', [0.0, 0.96, 0.87, 0.46, 0.26, 0.88, 0.94]], ['ConnectionEndFrequency', [0.0, 0.19, 1.0, 0.23, 0.94, 0.35, 0.97]], ['ConnectionLossFrequency', [0.0, 0.54, 0.78]], ['GroupEndFrequency', [0.0, 1.0, 0.11, 0.79]], ['GroupWinFrequency', [0.0, 0.11, 1.0]], ['LoopEndFrequency', [0.0, 0.14, 0.66]], ['LoopWinFrequency', [0.0, 0.14, 0.66]], ['PatternEndFrequency', [0.0, 0.63, 0.35]], ['PatternWinFrequency', [0.0, 0.63, 0.35]], ['NoTargetPieceWinFrequency', [0.0, 0.72, 0.77, 0.95, 0.32, 1.0]], ['EliminatePiecesLossFrequency', [0.0, 0.85, 0.96, 0.68]], ['EliminatePiecesDrawFrequency', [0.0, 0.03, 0.91, 1.0, 0.36, 0.86]], ['NoOwnPiecesLossFrequency', [0.0, 1.0, 0.68]], ['FillEndFrequency', [0.0, 1.0, 0.04, 0.01, 0.99, 0.72]], ['FillWinFrequency', [0.0, 1.0, 0.04, 0.01, 0.99]], ['ReachDrawFrequency', [0.0, 0.9, 0.98]], ['ScoringLossFrequency', [0.0, 0.6, 0.62]], ['NoMovesLossFrequency', [0.0, 1.0, 0.13, 0.06]], ['NoMovesDrawFrequency', [0.0, 0.01, 0.04, 0.03, 0.22]], ['BoardSitesOccupiedChangeNumTimes', [0.0, 0.06, 0.42, 0.12, 0.14, 0.94]], ['BranchingFactorChangeNumTimesn', [0.0, 0.3, 0.02, 0.07, 0.04, 0.13, 0.01, 0.21, 0.03]], ['PieceNumberChangeNumTimes', [0.0, 0.06, 0.42, 0.12, 0.14, 1.0]], ['selection1', ['ProgressiveHistory', 'UCB1', 'UCB1GRAVE', 'UCB1Tuned']], ['selection2', ['ProgressiveHistory', 'UCB1GRAVE', 'UCB1', 'UCB1Tuned']], ['exploration_const1', ['0.1', '0.6', '1.41421356237']], ['exploration_const2', ['0.6', '0.1', '1.41421356237']], ['playout1', ['MAST', 'NST', 'Random200']], ['playout2', ['Random200', 'NST', 'MAST']]]
        for col,unique in onehot_cols:
            for u in unique:
                df[f'{col}_{u}']=(df[col]==u).astype(np.int8)
                
        
        str_cols=['agent1','agent2','EnglishRules', 'LudRules']
        print(f"len(str_cols):{len(str_cols)}")
        if mode=='train':
            for col in str_cols:
                df=self.clean(df,col)
                tfidf = TfidfVectorizer(max_features=600)
                tfidf_feats=tfidf.fit_transform(df[col]).toarray()
                for i in range(tfidf_feats.shape[1]):
                    df[f"{col}_tfidf_{i}"]=tfidf_feats[:,i]
                self.tfidfs.append(tfidf)
                df[f'{col}_len']=df[col].apply(len)
        else:#mode=='test'
            for i in range(len(self.tfidfs)):
                df=self.clean(df,str_cols[i])
                tfidf=self.tfidfs[i]
                tfidf_feats=tfidf.transform(df[str_cols[i]]).toarray()
                for j in range(tfidf_feats.shape[1]):
                    df[f"{str_cols[i]}_tfidf_{j}"]=tfidf_feats[:,j]
                df[f'{str_cols[i]}_len']=df[str_cols[i]].apply(len)
        
        #模型筛选出的无用特征,出现频率少于1%
        drop_cols=['Cooperation', 'Team', 'TriangleShape', 'DiamondShape', 'SpiralShape', 'StarShape', 'SquarePyramidalShape', 'SemiRegularTiling', 'CircleTiling', 'SpiralTiling', 'MancalaThreeRows', 'MancalaSixRows', 'MancalaCircular', 'AlquerqueBoardWithOneTriangle', 'AlquerqueBoardWithTwoTriangles', 'AlquerqueBoardWithFourTriangles', 'AlquerqueBoardWithEightTriangles', 'ThreeMensMorrisBoard', 'ThreeMensMorrisBoardWithTwoTriangles', 'NineMensMorrisBoard', 'StarBoard', 'PachisiBoard', 'Boardless', 'NumColumns', 'NumCorners', 'NumOffDiagonalDirections', 'NumLayers', 'NumCentreSites', 'NumConvexCorners', 'NumPhasesBoard', 'NumContainers', 'Piece', 'PieceValue', 'PieceRotation', 'PieceDirection', 'LargePiece', 'Tile', 'NumComponentsType', 'NumDice', 'OpeningContract', 'SwapOption', 'Repetition', 'TurnKo', 'PositionalSuperko', 'AutoMove', 'InitialRandomPlacement', 'InitialScore', 'InitialCost', 'Moves', 'VoteDecision', 'SwapPlayersDecision', 'SwapPlayersDecisionFrequency', 'ProposeDecision', 'ProposeDecisionFrequency', 'PromotionDecisionFrequency', 'RotationDecision', 'RotationDecisionFrequency', 'StepDecisionToFriend', 'StepDecisionToFriendFrequency', 'StepDecisionToEnemy', 'SlideDecisionToEnemy', 'SlideDecisionToEnemyFrequency', 'SlideDecisionToFriend', 'SlideDecisionToFriendFrequency', 'LeapDecision', 'LeapDecisionFrequency', 'LeapDecisionToEmpty', 'LeapDecisionToEmptyFrequency', 'LeapDecisionToEnemy', 'LeapDecisionToEnemyFrequency', 'HopDecisionFriendToEmpty', 'HopDecisionFriendToEmptyFrequency', 'HopDecisionFriendToFriendFrequency', 'HopDecisionEnemyToEnemy', 'HopDecisionEnemyToEnemyFrequency', 'HopDecisionFriendToEnemy', 'HopDecisionFriendToEnemyFrequency', 'FromToDecisionFrequency', 'FromToDecisionEnemy', 'FromToDecisionEnemyFrequency', 'FromToDecisionFriend', 'SwapPiecesDecision', 'SwapPiecesDecisionFrequency', 'ShootDecision', 'ShootDecisionFrequency', 'VoteEffect', 'SwapPlayersEffect', 'PassEffect', 'ProposeEffect', 'ProposeEffectFrequency', 'AddEffectFrequency', 'SowFrequency', 'SowCapture', 'SowCaptureFrequency', 'SowRemove', 'SowBacktracking', 'SowBacktrackingFrequency', 'SowProperties', 'SowOriginFirst', 'SowCCW', 'PromotionEffectFrequency', 'PushEffect', 'PushEffectFrequency', 'Flip', 'FlipFrequency', 'SetNextPlayer', 'SetValue', 'SetValueFrequency', 'SetCount', 'SetCountFrequency', 'SetRotation', 'SetRotationFrequency', 'StepEffect', 'SlideEffect', 'LeapEffect', 'ByDieMove', 'MaxDistance', 'ReplacementCaptureFrequency', 'HopCaptureMoreThanOne', 'DirectionCapture', 'DirectionCaptureFrequency', 'EncloseCaptureFrequency', 'CustodialCapture', 'CustodialCaptureFrequency', 'InterveneCapture', 'InterveneCaptureFrequency', 'SurroundCapture', 'SurroundCaptureFrequency', 'CaptureSequence', 'CaptureSequenceFrequency', 'Group', 'Loop', 'Pattern', 'PathExtent', 'Territory', 'Fill', 'CanNotMove', 'Threat', 'CountPiecesMoverComparison', 'ProgressCheck', 'RotationalDirection', 'SameLayerDirection', 'ForwardDirection', 'BackwardDirection', 'BackwardsDirection', 'LeftwardDirection', 'RightwardsDirection', 'LeftwardsDirection', 'ForwardLeftDirection', 'ForwardRightDirection', 'BackwardLeftDirection', 'BackwardRightDirection', 'SameDirection', 'OppositeDirection', 'NumPlayPhase', 'LineLoss', 'LineLossFrequency', 'LineDraw', 'ConnectionEnd', 'ConnectionEndFrequency', 'ConnectionWinFrequency', 'ConnectionLoss', 'ConnectionLossFrequency', 'GroupEnd', 'GroupEndFrequency', 'GroupWin', 'GroupWinFrequency', 'GroupLoss', 'GroupDraw', 'LoopEnd', 'LoopEndFrequency', 'LoopWin', 'LoopWinFrequency', 'LoopLoss', 'PatternEnd', 'PatternEndFrequency', 'PatternWin', 'PatternWinFrequency', 'PathExtentEnd', 'PathExtentWin', 'PathExtentLoss', 'TerritoryEnd', 'TerritoryWin', 'TerritoryWinFrequency', 'Checkmate', 'CheckmateWin', 'NoTargetPieceEndFrequency', 'NoTargetPieceWin', 'NoTargetPieceWinFrequency', 'EliminatePiecesLoss', 'EliminatePiecesLossFrequency', 'EliminatePiecesDraw', 'EliminatePiecesDrawFrequency', 'NoOwnPiecesEnd', 'NoOwnPiecesWin', 'NoOwnPiecesLoss', 'NoOwnPiecesLossFrequency', 'FillEnd', 'FillEndFrequency', 'FillWin', 'FillWinFrequency', 'ReachWin', 'ReachLoss', 'ReachLossFrequency', 'ReachDraw', 'ReachDrawFrequency', 'ScoringLoss', 'ScoringLossFrequency', 'ScoringDraw', 'NoMovesLoss', 'NoMovesDrawFrequency', 'NoProgressEnd', 'NoProgressEndFrequency', 'NoProgressDraw', 'NoProgressDrawFrequency', 'BoardCoverageFull', 'BoardSitesOccupiedChangeNumTimes', 'BranchingFactorChangeLineBestFit', 'BranchingFactorChangeNumTimesn', 'DecisionFactorChangeNumTimes', 'MoveDistanceChangeSign', 'MoveDistanceChangeLineBestFit', 'PieceNumberChangeNumTimes', 'PieceNumberMaxIncrease', 'ScoreDifferenceMedian', 'ScoreDifferenceVariance', 'ScoreDifferenceChangeAverage', 'ScoreDifferenceChangeSign', 'ScoreDifferenceChangeLineBestFit', 'Math', 'Division', 'Modulo', 'Absolute', 'Exponentiation', 'Minimum', 'Maximum', 'Even', 'Odd', 'Visual', 'GraphStyle', 'MancalaStyle', 'PenAndPaperStyle', 'ShibumiStyle', 'BackgammonStyle', 'JanggiStyle', 'XiangqiStyle', 'ShogiStyle', 'TableStyle', 'SurakartaStyle', 'NoBoard', 'ChessComponent', 'KingComponent', 'QueenComponent', 'KnightComponent', 'RookComponent', 'BishopComponent', 'PawnComponent', 'FairyChessComponent', 'PloyComponent', 'ShogiComponent', 'XiangqiComponent', 'StrategoComponent', 'JanggiComponent', 'TaflComponent', 'StackType', 'Stack', 'ShowPieceValue', 'ShowPieceState', 'Implementation', 'StateType', 'StackState', 'VisitedSites', 'InternalCounter', 'SetInternalCounter', 'Efficiency', 'NumOffDiagonalDirections_0.0', 'NumOffDiagonalDirections_4.82', 'NumOffDiagonalDirections_2.0', 'NumOffDiagonalDirections_5.18', 'NumOffDiagonalDirections_3.08', 'NumOffDiagonalDirections_0.06', 'NumLayers_1', 'NumLayers_0', 'NumLayers_4', 'NumLayers_5', 'NumPhasesBoard_1', 'NumPhasesBoard_5', 'NumDice_0', 'NumDice_2', 'NumDice_6', 'NumDice_3', 'NumDice_5', 'NumDice_7', 'ProposeDecisionFrequency_0.0', 'ProposeDecisionFrequency_0.05', 'ProposeDecisionFrequency_0.01', 'PromotionDecisionFrequency_0.0', 'PromotionDecisionFrequency_0.01', 'PromotionDecisionFrequency_0.03', 'PromotionDecisionFrequency_0.02', 'PromotionDecisionFrequency_0.11', 'PromotionDecisionFrequency_0.05', 'PromotionDecisionFrequency_0.04', 'SlideDecisionToFriendFrequency_0.0', 'SlideDecisionToFriendFrequency_0.19', 'SlideDecisionToFriendFrequency_0.06', 'LeapDecisionToEnemyFrequency_0.0', 'LeapDecisionToEnemyFrequency_0.04', 'LeapDecisionToEnemyFrequency_0.01', 'LeapDecisionToEnemyFrequency_0.02', 'LeapDecisionToEnemyFrequency_0.07', 'LeapDecisionToEnemyFrequency_0.03', 'LeapDecisionToEnemyFrequency_0.14', 'LeapDecisionToEnemyFrequency_0.08', 'HopDecisionFriendToFriendFrequency_0.0', 'HopDecisionFriendToFriendFrequency_0.13', 'HopDecisionFriendToFriendFrequency_0.09', 'HopDecisionEnemyToEnemyFrequency_0.0', 'HopDecisionEnemyToEnemyFrequency_0.01', 'HopDecisionEnemyToEnemyFrequency_0.2', 'HopDecisionEnemyToEnemyFrequency_0.03', 'HopDecisionFriendToEnemyFrequency_0.0', 'HopDecisionFriendToEnemyFrequency_0.01', 'HopDecisionFriendToEnemyFrequency_0.09', 'HopDecisionFriendToEnemyFrequency_0.25', 'HopDecisionFriendToEnemyFrequency_0.02', 'FromToDecisionFrequency_0.0', 'FromToDecisionFrequency_0.38', 'FromToDecisionFrequency_1.0', 'FromToDecisionFrequency_0.31', 'FromToDecisionFrequency_0.94', 'FromToDecisionFrequency_0.67', 'ProposeEffectFrequency_0.0', 'ProposeEffectFrequency_0.01', 'ProposeEffectFrequency_0.03', 'PushEffectFrequency_0.0', 'PushEffectFrequency_0.5', 'PushEffectFrequency_0.96', 'PushEffectFrequency_0.25', 'FlipFrequency_0.0', 'FlipFrequency_0.87', 'FlipFrequency_1.0', 'FlipFrequency_0.96', 'SetCountFrequency_0.0', 'SetCountFrequency_0.62', 'SetCountFrequency_0.54', 'SetCountFrequency_0.02', 'DirectionCaptureFrequency_0.0', 'DirectionCaptureFrequency_0.55', 'DirectionCaptureFrequency_0.54', 'EncloseCaptureFrequency_0.0', 'EncloseCaptureFrequency_0.08', 'EncloseCaptureFrequency_0.1', 'EncloseCaptureFrequency_0.07', 'EncloseCaptureFrequency_0.12', 'EncloseCaptureFrequency_0.02', 'EncloseCaptureFrequency_0.09', 'InterveneCaptureFrequency_0.0', 'InterveneCaptureFrequency_0.01', 'InterveneCaptureFrequency_0.14', 'InterveneCaptureFrequency_0.04', 'SurroundCaptureFrequency_0.0', 'SurroundCaptureFrequency_0.01', 'SurroundCaptureFrequency_0.03', 'SurroundCaptureFrequency_0.02', 'NumPlayPhase_3', 'NumPlayPhase_4', 'NumPlayPhase_5', 'NumPlayPhase_6', 'NumPlayPhase_7', 'NumPlayPhase_8', 'LineLossFrequency_0.0', 'LineLossFrequency_0.96', 'LineLossFrequency_0.87', 'LineLossFrequency_0.46', 'LineLossFrequency_0.26', 'LineLossFrequency_0.88', 'LineLossFrequency_0.94', 'ConnectionEndFrequency_0.0', 'ConnectionEndFrequency_0.19', 'ConnectionEndFrequency_1.0', 'ConnectionEndFrequency_0.23', 'ConnectionEndFrequency_0.94', 'ConnectionEndFrequency_0.35', 'ConnectionEndFrequency_0.97', 'ConnectionLossFrequency_0.0', 'ConnectionLossFrequency_0.54', 'ConnectionLossFrequency_0.78', 'GroupEndFrequency_0.0', 'GroupEndFrequency_1.0', 'GroupEndFrequency_0.11', 'GroupEndFrequency_0.79', 'GroupWinFrequency_0.0', 'GroupWinFrequency_0.11', 'GroupWinFrequency_1.0', 'LoopEndFrequency_0.0', 'LoopEndFrequency_0.14', 'LoopEndFrequency_0.66', 'LoopWinFrequency_0.0', 'LoopWinFrequency_0.14', 'LoopWinFrequency_0.66', 'PatternEndFrequency_0.0', 'PatternEndFrequency_0.63', 'PatternEndFrequency_0.35', 'PatternWinFrequency_0.0', 'PatternWinFrequency_0.63', 'PatternWinFrequency_0.35', 'NoTargetPieceWinFrequency_0.0', 'NoTargetPieceWinFrequency_0.72', 'NoTargetPieceWinFrequency_0.77', 'NoTargetPieceWinFrequency_0.95', 'NoTargetPieceWinFrequency_0.32', 'NoTargetPieceWinFrequency_1.0', 'EliminatePiecesLossFrequency_0.0', 'EliminatePiecesLossFrequency_0.85', 'EliminatePiecesLossFrequency_0.96', 'EliminatePiecesLossFrequency_0.68', 'EliminatePiecesDrawFrequency_0.0', 'EliminatePiecesDrawFrequency_0.03', 'EliminatePiecesDrawFrequency_0.91', 'EliminatePiecesDrawFrequency_1.0', 'EliminatePiecesDrawFrequency_0.36', 'EliminatePiecesDrawFrequency_0.86', 'NoOwnPiecesLossFrequency_0.0', 'NoOwnPiecesLossFrequency_1.0', 'NoOwnPiecesLossFrequency_0.68', 'FillEndFrequency_0.0', 'FillEndFrequency_1.0', 'FillEndFrequency_0.04', 'FillEndFrequency_0.01', 'FillEndFrequency_0.99', 'FillEndFrequency_0.72', 'FillWinFrequency_0.0', 'FillWinFrequency_1.0', 'FillWinFrequency_0.04', 'FillWinFrequency_0.01', 'FillWinFrequency_0.99', 'ReachDrawFrequency_0.0', 'ReachDrawFrequency_0.9', 'ReachDrawFrequency_0.98', 'ScoringLossFrequency_0.0', 'ScoringLossFrequency_0.6', 'ScoringLossFrequency_0.62', 'NoMovesLossFrequency_0.0', 'NoMovesLossFrequency_1.0', 'NoMovesLossFrequency_0.13', 'NoMovesLossFrequency_0.06', 'NoMovesDrawFrequency_0.0', 'NoMovesDrawFrequency_0.01', 'NoMovesDrawFrequency_0.04', 'NoMovesDrawFrequency_0.03', 'NoMovesDrawFrequency_0.22', 'BoardSitesOccupiedChangeNumTimes_0.0', 'BoardSitesOccupiedChangeNumTimes_0.06', 'BoardSitesOccupiedChangeNumTimes_0.42', 'BoardSitesOccupiedChangeNumTimes_0.12', 'BoardSitesOccupiedChangeNumTimes_0.14', 'BoardSitesOccupiedChangeNumTimes_0.94', 'BranchingFactorChangeNumTimesn_0.0', 'BranchingFactorChangeNumTimesn_0.3', 'BranchingFactorChangeNumTimesn_0.02', 'BranchingFactorChangeNumTimesn_0.07', 'BranchingFactorChangeNumTimesn_0.04', 'BranchingFactorChangeNumTimesn_0.13', 'BranchingFactorChangeNumTimesn_0.01', 'BranchingFactorChangeNumTimesn_0.21', 'BranchingFactorChangeNumTimesn_0.03', 'PieceNumberChangeNumTimes_0.0', 'PieceNumberChangeNumTimes_0.06', 'PieceNumberChangeNumTimes_0.42', 'PieceNumberChangeNumTimes_0.12', 'PieceNumberChangeNumTimes_0.14', 'PieceNumberChangeNumTimes_1.0', 'EnglishRules_tfidf_0', 'EnglishRules_tfidf_1', 'EnglishRules_tfidf_3', 'EnglishRules_tfidf_4', 'EnglishRules_tfidf_12', 'EnglishRules_tfidf_13', 'EnglishRules_tfidf_14', 'EnglishRules_tfidf_15', 'EnglishRules_tfidf_16', 'EnglishRules_tfidf_17', 'EnglishRules_tfidf_20', 'EnglishRules_tfidf_23', 'EnglishRules_tfidf_25', 'EnglishRules_tfidf_28', 'EnglishRules_tfidf_29', 'EnglishRules_tfidf_33', 'EnglishRules_tfidf_35', 'EnglishRules_tfidf_36', 'EnglishRules_tfidf_38', 'EnglishRules_tfidf_39', 'EnglishRules_tfidf_40', 'EnglishRules_tfidf_44', 'EnglishRules_tfidf_46', 'EnglishRules_tfidf_48', 'EnglishRules_tfidf_50', 'EnglishRules_tfidf_51', 'EnglishRules_tfidf_52', 'EnglishRules_tfidf_58', 'EnglishRules_tfidf_59', 'EnglishRules_tfidf_62', 'EnglishRules_tfidf_64', 'EnglishRules_tfidf_65', 'EnglishRules_tfidf_66', 'EnglishRules_tfidf_68', 'EnglishRules_tfidf_73', 'EnglishRules_tfidf_76', 'EnglishRules_tfidf_83', 'EnglishRules_tfidf_84', 'EnglishRules_tfidf_85', 'EnglishRules_tfidf_86', 'EnglishRules_tfidf_87', 'EnglishRules_tfidf_88', 'EnglishRules_tfidf_89', 'EnglishRules_tfidf_90', 'EnglishRules_tfidf_91', 'EnglishRules_tfidf_92', 'EnglishRules_tfidf_93', 'EnglishRules_tfidf_94', 'EnglishRules_tfidf_95', 'EnglishRules_tfidf_96', 'EnglishRules_tfidf_97', 'EnglishRules_tfidf_98', 'EnglishRules_tfidf_99', 'EnglishRules_tfidf_100', 'EnglishRules_tfidf_101', 'EnglishRules_tfidf_105', 'EnglishRules_tfidf_106', 'EnglishRules_tfidf_107', 'EnglishRules_tfidf_108', 'EnglishRules_tfidf_109', 'EnglishRules_tfidf_113', 'EnglishRules_tfidf_114', 'EnglishRules_tfidf_116', 'EnglishRules_tfidf_117', 'EnglishRules_tfidf_118', 'EnglishRules_tfidf_119', 'EnglishRules_tfidf_120', 'EnglishRules_tfidf_121', 'EnglishRules_tfidf_122', 'EnglishRules_tfidf_123', 'EnglishRules_tfidf_124', 'EnglishRules_tfidf_125', 'EnglishRules_tfidf_126', 'EnglishRules_tfidf_127', 'EnglishRules_tfidf_128', 'EnglishRules_tfidf_129', 'EnglishRules_tfidf_130', 'EnglishRules_tfidf_131', 'EnglishRules_tfidf_132', 'EnglishRules_tfidf_133', 'EnglishRules_tfidf_134', 'EnglishRules_tfidf_135', 'EnglishRules_tfidf_136', 'EnglishRules_tfidf_137', 'EnglishRules_tfidf_138', 'EnglishRules_tfidf_139', 'EnglishRules_tfidf_140', 'EnglishRules_tfidf_142', 'EnglishRules_tfidf_143', 'EnglishRules_tfidf_144', 'EnglishRules_tfidf_145', 'EnglishRules_tfidf_146', 'EnglishRules_tfidf_147', 'EnglishRules_tfidf_148', 'EnglishRules_tfidf_150', 'EnglishRules_tfidf_151', 'EnglishRules_tfidf_152', 'EnglishRules_tfidf_153', 'EnglishRules_tfidf_154', 'EnglishRules_tfidf_155', 'EnglishRules_tfidf_156', 'EnglishRules_tfidf_158', 'EnglishRules_tfidf_161', 'EnglishRules_tfidf_162', 'EnglishRules_tfidf_164', 'EnglishRules_tfidf_166', 'EnglishRules_tfidf_170', 'EnglishRules_tfidf_171', 'EnglishRules_tfidf_172', 'EnglishRules_tfidf_174', 'EnglishRules_tfidf_176', 'EnglishRules_tfidf_177', 'EnglishRules_tfidf_178', 'EnglishRules_tfidf_179', 'EnglishRules_tfidf_180', 'EnglishRules_tfidf_181', 'EnglishRules_tfidf_182', 'EnglishRules_tfidf_183', 'EnglishRules_tfidf_184', 'EnglishRules_tfidf_185', 'EnglishRules_tfidf_186', 'EnglishRules_tfidf_188', 'EnglishRules_tfidf_190', 'EnglishRules_tfidf_192', 'EnglishRules_tfidf_195', 'EnglishRules_tfidf_196', 'EnglishRules_tfidf_198', 'EnglishRules_tfidf_199', 'EnglishRules_tfidf_200', 'EnglishRules_tfidf_201', 'EnglishRules_tfidf_203', 'EnglishRules_tfidf_204', 'EnglishRules_tfidf_205', 'EnglishRules_tfidf_207', 'EnglishRules_tfidf_208', 'EnglishRules_tfidf_209', 'EnglishRules_tfidf_210', 'EnglishRules_tfidf_211', 'EnglishRules_tfidf_213', 'EnglishRules_tfidf_214', 'EnglishRules_tfidf_215', 'EnglishRules_tfidf_217', 'EnglishRules_tfidf_218', 'EnglishRules_tfidf_219', 'EnglishRules_tfidf_220', 'EnglishRules_tfidf_221', 'EnglishRules_tfidf_223', 'EnglishRules_tfidf_224', 'EnglishRules_tfidf_225', 'EnglishRules_tfidf_227', 'EnglishRules_tfidf_230', 'EnglishRules_tfidf_232', 'EnglishRules_tfidf_233', 'EnglishRules_tfidf_234', 'EnglishRules_tfidf_236', 'EnglishRules_tfidf_237', 'EnglishRules_tfidf_241', 'EnglishRules_tfidf_242', 'EnglishRules_tfidf_244', 'EnglishRules_tfidf_245', 'EnglishRules_tfidf_246', 'EnglishRules_tfidf_247', 'EnglishRules_tfidf_248', 'EnglishRules_tfidf_249', 'EnglishRules_tfidf_252', 'EnglishRules_tfidf_254', 'EnglishRules_tfidf_255', 'EnglishRules_tfidf_256', 'EnglishRules_tfidf_257', 'EnglishRules_tfidf_258', 'EnglishRules_tfidf_260', 'EnglishRules_tfidf_262', 'EnglishRules_tfidf_267', 'EnglishRules_tfidf_268', 'EnglishRules_tfidf_269', 'EnglishRules_tfidf_270', 'EnglishRules_tfidf_271', 'EnglishRules_tfidf_272', 'EnglishRules_tfidf_274', 'EnglishRules_tfidf_275', 'EnglishRules_tfidf_276', 'EnglishRules_tfidf_277', 'EnglishRules_tfidf_279', 'EnglishRules_tfidf_280', 'EnglishRules_tfidf_281', 'EnglishRules_tfidf_283', 'EnglishRules_tfidf_284', 'EnglishRules_tfidf_285', 'EnglishRules_tfidf_287', 'EnglishRules_tfidf_289', 'EnglishRules_tfidf_290', 'EnglishRules_tfidf_294', 'EnglishRules_tfidf_295', 'EnglishRules_tfidf_296', 'EnglishRules_tfidf_297', 'EnglishRules_tfidf_300', 'EnglishRules_tfidf_303', 'EnglishRules_tfidf_304', 'EnglishRules_tfidf_305', 'EnglishRules_tfidf_306', 'EnglishRules_tfidf_308', 'EnglishRules_tfidf_310', 'EnglishRules_tfidf_311', 'EnglishRules_tfidf_312', 'EnglishRules_tfidf_317', 'EnglishRules_tfidf_318', 'EnglishRules_tfidf_319', 'EnglishRules_tfidf_320', 'EnglishRules_tfidf_322', 'EnglishRules_tfidf_324', 'EnglishRules_tfidf_325', 'EnglishRules_tfidf_326', 'EnglishRules_tfidf_327', 'EnglishRules_tfidf_328', 'EnglishRules_tfidf_329', 'EnglishRules_tfidf_331', 'EnglishRules_tfidf_332', 'EnglishRules_tfidf_334', 'EnglishRules_tfidf_338', 'EnglishRules_tfidf_340', 'EnglishRules_tfidf_344', 'EnglishRules_tfidf_348', 'EnglishRules_tfidf_350', 'EnglishRules_tfidf_352', 'EnglishRules_tfidf_353', 'EnglishRules_tfidf_354', 'EnglishRules_tfidf_356', 'EnglishRules_tfidf_357', 'EnglishRules_tfidf_361', 'EnglishRules_tfidf_363', 'EnglishRules_tfidf_366', 'EnglishRules_tfidf_367', 'EnglishRules_tfidf_368', 'EnglishRules_tfidf_369', 'EnglishRules_tfidf_370', 'EnglishRules_tfidf_371', 'EnglishRules_tfidf_372', 'EnglishRules_tfidf_373', 'EnglishRules_tfidf_375', 'EnglishRules_tfidf_376', 'EnglishRules_tfidf_378', 'EnglishRules_tfidf_379', 'EnglishRules_tfidf_380', 'EnglishRules_tfidf_381', 'EnglishRules_tfidf_385', 'EnglishRules_tfidf_387', 'EnglishRules_tfidf_388', 'EnglishRules_tfidf_395', 'EnglishRules_tfidf_397', 'EnglishRules_tfidf_400', 'EnglishRules_tfidf_401', 'EnglishRules_tfidf_403', 'EnglishRules_tfidf_404', 'EnglishRules_tfidf_406', 'EnglishRules_tfidf_407', 'EnglishRules_tfidf_408', 'EnglishRules_tfidf_409', 'EnglishRules_tfidf_410', 'EnglishRules_tfidf_411', 'EnglishRules_tfidf_412', 'EnglishRules_tfidf_413', 'EnglishRules_tfidf_414', 'EnglishRules_tfidf_415', 'EnglishRules_tfidf_416', 'EnglishRules_tfidf_417', 'EnglishRules_tfidf_420', 'EnglishRules_tfidf_421', 'EnglishRules_tfidf_422', 'EnglishRules_tfidf_423', 'EnglishRules_tfidf_424', 'EnglishRules_tfidf_425', 'EnglishRules_tfidf_427', 'EnglishRules_tfidf_428', 'EnglishRules_tfidf_430', 'EnglishRules_tfidf_431', 'EnglishRules_tfidf_432', 'EnglishRules_tfidf_433', 'EnglishRules_tfidf_434', 'EnglishRules_tfidf_436', 'EnglishRules_tfidf_437', 'EnglishRules_tfidf_438', 'EnglishRules_tfidf_439', 'EnglishRules_tfidf_440', 'EnglishRules_tfidf_442', 'EnglishRules_tfidf_445', 'EnglishRules_tfidf_449', 'EnglishRules_tfidf_450', 'EnglishRules_tfidf_452', 'EnglishRules_tfidf_453', 'EnglishRules_tfidf_454', 'EnglishRules_tfidf_456', 'EnglishRules_tfidf_457', 'EnglishRules_tfidf_458', 'EnglishRules_tfidf_459', 'EnglishRules_tfidf_460', 'EnglishRules_tfidf_461', 'EnglishRules_tfidf_463', 'EnglishRules_tfidf_464', 'EnglishRules_tfidf_465', 'EnglishRules_tfidf_469', 'EnglishRules_tfidf_470', 'EnglishRules_tfidf_472', 'EnglishRules_tfidf_473', 'EnglishRules_tfidf_475', 'EnglishRules_tfidf_476', 'EnglishRules_tfidf_477', 'EnglishRules_tfidf_478', 'EnglishRules_tfidf_479', 'EnglishRules_tfidf_481', 'EnglishRules_tfidf_482', 'EnglishRules_tfidf_483', 'EnglishRules_tfidf_484', 'EnglishRules_tfidf_485', 'EnglishRules_tfidf_486', 'EnglishRules_tfidf_487', 'EnglishRules_tfidf_488', 'EnglishRules_tfidf_491', 'EnglishRules_tfidf_493', 'EnglishRules_tfidf_494', 'EnglishRules_tfidf_495', 'EnglishRules_tfidf_496', 'EnglishRules_tfidf_497', 'EnglishRules_tfidf_500', 'EnglishRules_tfidf_503', 'EnglishRules_tfidf_504', 'EnglishRules_tfidf_505', 'EnglishRules_tfidf_508', 'EnglishRules_tfidf_509', 'EnglishRules_tfidf_510', 'EnglishRules_tfidf_511', 'EnglishRules_tfidf_512', 'EnglishRules_tfidf_513', 'EnglishRules_tfidf_514', 'EnglishRules_tfidf_515', 'EnglishRules_tfidf_519', 'EnglishRules_tfidf_520', 'EnglishRules_tfidf_521', 'EnglishRules_tfidf_529', 'EnglishRules_tfidf_531', 'EnglishRules_tfidf_533', 'EnglishRules_tfidf_536', 'EnglishRules_tfidf_537', 'EnglishRules_tfidf_538', 'EnglishRules_tfidf_540', 'EnglishRules_tfidf_541', 'EnglishRules_tfidf_542', 'EnglishRules_tfidf_543', 'EnglishRules_tfidf_545', 'EnglishRules_tfidf_546', 'EnglishRules_tfidf_547', 'EnglishRules_tfidf_548', 'EnglishRules_tfidf_549', 'EnglishRules_tfidf_554', 'EnglishRules_tfidf_555', 'EnglishRules_tfidf_557', 'EnglishRules_tfidf_558', 'EnglishRules_tfidf_559', 'EnglishRules_tfidf_560', 'EnglishRules_tfidf_564', 'EnglishRules_tfidf_565', 'EnglishRules_tfidf_566', 'EnglishRules_tfidf_567', 'EnglishRules_tfidf_568', 'EnglishRules_tfidf_570', 'EnglishRules_tfidf_571', 'EnglishRules_tfidf_573', 'EnglishRules_tfidf_574', 'EnglishRules_tfidf_576', 'EnglishRules_tfidf_577', 'EnglishRules_tfidf_581', 'EnglishRules_tfidf_584', 'EnglishRules_tfidf_588', 'EnglishRules_tfidf_591', 'EnglishRules_tfidf_592', 'EnglishRules_tfidf_593', 'EnglishRules_tfidf_594', 'EnglishRules_tfidf_595', 'EnglishRules_tfidf_596', 'EnglishRules_tfidf_597', 'EnglishRules_tfidf_598', 'LudRules_tfidf_1', 'LudRules_tfidf_2', 'LudRules_tfidf_9', 'LudRules_tfidf_11', 'LudRules_tfidf_16', 'LudRules_tfidf_17', 'LudRules_tfidf_19', 'LudRules_tfidf_21', 'LudRules_tfidf_23', 'LudRules_tfidf_24', 'LudRules_tfidf_25', 'LudRules_tfidf_26', 'LudRules_tfidf_28', 'LudRules_tfidf_29', 'LudRules_tfidf_30', 'LudRules_tfidf_31', 'LudRules_tfidf_32', 'LudRules_tfidf_33', 'LudRules_tfidf_34', 'LudRules_tfidf_35', 'LudRules_tfidf_36', 'LudRules_tfidf_37', 'LudRules_tfidf_39', 'LudRules_tfidf_40', 'LudRules_tfidf_41', 'LudRules_tfidf_42', 'LudRules_tfidf_44', 'LudRules_tfidf_45', 'LudRules_tfidf_46', 'LudRules_tfidf_47', 'LudRules_tfidf_49', 'LudRules_tfidf_50', 'LudRules_tfidf_51', 'LudRules_tfidf_53', 'LudRules_tfidf_55', 'LudRules_tfidf_56', 'LudRules_tfidf_57', 'LudRules_tfidf_58', 'LudRules_tfidf_59', 'LudRules_tfidf_60', 'LudRules_tfidf_61', 'LudRules_tfidf_63', 'LudRules_tfidf_64', 'LudRules_tfidf_65', 'LudRules_tfidf_66', 'LudRules_tfidf_67', 'LudRules_tfidf_68', 'LudRules_tfidf_69', 'LudRules_tfidf_70', 'LudRules_tfidf_71', 'LudRules_tfidf_73', 'LudRules_tfidf_74', 'LudRules_tfidf_75', 'LudRules_tfidf_76', 'LudRules_tfidf_77', 'LudRules_tfidf_78', 'LudRules_tfidf_79', 'LudRules_tfidf_80', 'LudRules_tfidf_82', 'LudRules_tfidf_83', 'LudRules_tfidf_84', 'LudRules_tfidf_86', 'LudRules_tfidf_89', 'LudRules_tfidf_91', 'LudRules_tfidf_94', 'LudRules_tfidf_95', 'LudRules_tfidf_99', 'LudRules_tfidf_100', 'LudRules_tfidf_101', 'LudRules_tfidf_102', 'LudRules_tfidf_103', 'LudRules_tfidf_104', 'LudRules_tfidf_105', 'LudRules_tfidf_106', 'LudRules_tfidf_107', 'LudRules_tfidf_108', 'LudRules_tfidf_109', 'LudRules_tfidf_110', 'LudRules_tfidf_111', 'LudRules_tfidf_113', 'LudRules_tfidf_114', 'LudRules_tfidf_115', 'LudRules_tfidf_116', 'LudRules_tfidf_117', 'LudRules_tfidf_118', 'LudRules_tfidf_121', 'LudRules_tfidf_122', 'LudRules_tfidf_123', 'LudRules_tfidf_126', 'LudRules_tfidf_127', 'LudRules_tfidf_128', 'LudRules_tfidf_129', 'LudRules_tfidf_130', 'LudRules_tfidf_131', 'LudRules_tfidf_132', 'LudRules_tfidf_134', 'LudRules_tfidf_135', 'LudRules_tfidf_136', 'LudRules_tfidf_137', 'LudRules_tfidf_138', 'LudRules_tfidf_140', 'LudRules_tfidf_141', 'LudRules_tfidf_142', 'LudRules_tfidf_143', 'LudRules_tfidf_144', 'LudRules_tfidf_146', 'LudRules_tfidf_147', 'LudRules_tfidf_148', 'LudRules_tfidf_149', 'LudRules_tfidf_150', 'LudRules_tfidf_152', 'LudRules_tfidf_155', 'LudRules_tfidf_156', 'LudRules_tfidf_159', 'LudRules_tfidf_161', 'LudRules_tfidf_164', 'LudRules_tfidf_166', 'LudRules_tfidf_167', 'LudRules_tfidf_168', 'LudRules_tfidf_169', 'LudRules_tfidf_170', 'LudRules_tfidf_173', 'LudRules_tfidf_174', 'LudRules_tfidf_176', 'LudRules_tfidf_178', 'LudRules_tfidf_179', 'LudRules_tfidf_182', 'LudRules_tfidf_183', 'LudRules_tfidf_184', 'LudRules_tfidf_185', 'LudRules_tfidf_187', 'LudRules_tfidf_189', 'LudRules_tfidf_190', 'LudRules_tfidf_192', 'LudRules_tfidf_193', 'LudRules_tfidf_194', 'LudRules_tfidf_195', 'LudRules_tfidf_197', 'LudRules_tfidf_198', 'LudRules_tfidf_199', 'LudRules_tfidf_200', 'LudRules_tfidf_202', 'LudRules_tfidf_204', 'LudRules_tfidf_205', 'LudRules_tfidf_206', 'LudRules_tfidf_208', 'LudRules_tfidf_210', 'LudRules_tfidf_211', 'LudRules_tfidf_212', 'LudRules_tfidf_214', 'LudRules_tfidf_216', 'LudRules_tfidf_217', 'LudRules_tfidf_218', 'LudRules_tfidf_220', 'LudRules_tfidf_222', 'LudRules_tfidf_224', 'LudRules_tfidf_225', 'LudRules_tfidf_226', 'LudRules_tfidf_227', 'LudRules_tfidf_228', 'LudRules_tfidf_229', 'LudRules_tfidf_230', 'LudRules_tfidf_231', 'LudRules_tfidf_232', 'LudRules_tfidf_234', 'LudRules_tfidf_235', 'LudRules_tfidf_236', 'LudRules_tfidf_237', 'LudRules_tfidf_239', 'LudRules_tfidf_240', 'LudRules_tfidf_241', 'LudRules_tfidf_243', 'LudRules_tfidf_247', 'LudRules_tfidf_248', 'LudRules_tfidf_250', 'LudRules_tfidf_251', 'LudRules_tfidf_252', 'LudRules_tfidf_253', 'LudRules_tfidf_255', 'LudRules_tfidf_257', 'LudRules_tfidf_258', 'LudRules_tfidf_259', 'LudRules_tfidf_260', 'LudRules_tfidf_261', 'LudRules_tfidf_262', 'LudRules_tfidf_263', 'LudRules_tfidf_266', 'LudRules_tfidf_267', 'LudRules_tfidf_268', 'LudRules_tfidf_269', 'LudRules_tfidf_270', 'LudRules_tfidf_271', 'LudRules_tfidf_272', 'LudRules_tfidf_277', 'LudRules_tfidf_280', 'LudRules_tfidf_281', 'LudRules_tfidf_282', 'LudRules_tfidf_283', 'LudRules_tfidf_286', 'LudRules_tfidf_289', 'LudRules_tfidf_294', 'LudRules_tfidf_295', 'LudRules_tfidf_296', 'LudRules_tfidf_297', 'LudRules_tfidf_298', 'LudRules_tfidf_299', 'LudRules_tfidf_300', 'LudRules_tfidf_302', 'LudRules_tfidf_303', 'LudRules_tfidf_304', 'LudRules_tfidf_305', 'LudRules_tfidf_306', 'LudRules_tfidf_307', 'LudRules_tfidf_308', 'LudRules_tfidf_309', 'LudRules_tfidf_310', 'LudRules_tfidf_311', 'LudRules_tfidf_312', 'LudRules_tfidf_313', 'LudRules_tfidf_314', 'LudRules_tfidf_315', 'LudRules_tfidf_316', 'LudRules_tfidf_317', 'LudRules_tfidf_318', 'LudRules_tfidf_319', 'LudRules_tfidf_320', 'LudRules_tfidf_321', 'LudRules_tfidf_322', 'LudRules_tfidf_324', 'LudRules_tfidf_325', 'LudRules_tfidf_326', 'LudRules_tfidf_327', 'LudRules_tfidf_328', 'LudRules_tfidf_329', 'LudRules_tfidf_330', 'LudRules_tfidf_332', 'LudRules_tfidf_333', 'LudRules_tfidf_335', 'LudRules_tfidf_337', 'LudRules_tfidf_338', 'LudRules_tfidf_339', 'LudRules_tfidf_341', 'LudRules_tfidf_342', 'LudRules_tfidf_343', 'LudRules_tfidf_348', 'LudRules_tfidf_351', 'LudRules_tfidf_353', 'LudRules_tfidf_354', 'LudRules_tfidf_357', 'LudRules_tfidf_362', 'LudRules_tfidf_363', 'LudRules_tfidf_364', 'LudRules_tfidf_365', 'LudRules_tfidf_366', 'LudRules_tfidf_367', 'LudRules_tfidf_368', 'LudRules_tfidf_370', 'LudRules_tfidf_371', 'LudRules_tfidf_372', 'LudRules_tfidf_373', 'LudRules_tfidf_375', 'LudRules_tfidf_376', 'LudRules_tfidf_377', 'LudRules_tfidf_378', 'LudRules_tfidf_379', 'LudRules_tfidf_381', 'LudRules_tfidf_383', 'LudRules_tfidf_384', 'LudRules_tfidf_385', 'LudRules_tfidf_386', 'LudRules_tfidf_388', 'LudRules_tfidf_391', 'LudRules_tfidf_393', 'LudRules_tfidf_394', 'LudRules_tfidf_395', 'LudRules_tfidf_397', 'LudRules_tfidf_399', 'LudRules_tfidf_400', 'LudRules_tfidf_401', 'LudRules_tfidf_402', 'LudRules_tfidf_403', 'LudRules_tfidf_404', 'LudRules_tfidf_409', 'LudRules_tfidf_410', 'LudRules_tfidf_411', 'LudRules_tfidf_412', 'LudRules_tfidf_413', 'LudRules_tfidf_417', 'LudRules_tfidf_420', 'LudRules_tfidf_421', 'LudRules_tfidf_422', 'LudRules_tfidf_426', 'LudRules_tfidf_430', 'LudRules_tfidf_433', 'LudRules_tfidf_434', 'LudRules_tfidf_435', 'LudRules_tfidf_436', 'LudRules_tfidf_437', 'LudRules_tfidf_438', 'LudRules_tfidf_439', 'LudRules_tfidf_440', 'LudRules_tfidf_441', 'LudRules_tfidf_442', 'LudRules_tfidf_443', 'LudRules_tfidf_444', 'LudRules_tfidf_445', 'LudRules_tfidf_446', 'LudRules_tfidf_448', 'LudRules_tfidf_449', 'LudRules_tfidf_450', 'LudRules_tfidf_451', 'LudRules_tfidf_453', 'LudRules_tfidf_454', 'LudRules_tfidf_456', 'LudRules_tfidf_458', 'LudRules_tfidf_461', 'LudRules_tfidf_462', 'LudRules_tfidf_463', 'LudRules_tfidf_465', 'LudRules_tfidf_466', 'LudRules_tfidf_467', 'LudRules_tfidf_468', 'LudRules_tfidf_469', 'LudRules_tfidf_470', 'LudRules_tfidf_471', 'LudRules_tfidf_472', 'LudRules_tfidf_476', 'LudRules_tfidf_477', 'LudRules_tfidf_478', 'LudRules_tfidf_479', 'LudRules_tfidf_480', 'LudRules_tfidf_481', 'LudRules_tfidf_482', 'LudRules_tfidf_483', 'LudRules_tfidf_484', 'LudRules_tfidf_485', 'LudRules_tfidf_486', 'LudRules_tfidf_487', 'LudRules_tfidf_488', 'LudRules_tfidf_489', 'LudRules_tfidf_492', 'LudRules_tfidf_496', 'LudRules_tfidf_500', 'LudRules_tfidf_501', 'LudRules_tfidf_503', 'LudRules_tfidf_506', 'LudRules_tfidf_509', 'LudRules_tfidf_510', 'LudRules_tfidf_511', 'LudRules_tfidf_513', 'LudRules_tfidf_514', 'LudRules_tfidf_515', 'LudRules_tfidf_516', 'LudRules_tfidf_517', 'LudRules_tfidf_518', 'LudRules_tfidf_521', 'LudRules_tfidf_522', 'LudRules_tfidf_523', 'LudRules_tfidf_524', 'LudRules_tfidf_527', 'LudRules_tfidf_528', 'LudRules_tfidf_529', 'LudRules_tfidf_530', 'LudRules_tfidf_531', 'LudRules_tfidf_532', 'LudRules_tfidf_533', 'LudRules_tfidf_535', 'LudRules_tfidf_536', 'LudRules_tfidf_537', 'LudRules_tfidf_539', 'LudRules_tfidf_540', 'LudRules_tfidf_541', 'LudRules_tfidf_542', 'LudRules_tfidf_543', 'LudRules_tfidf_544', 'LudRules_tfidf_545', 'LudRules_tfidf_546', 'LudRules_tfidf_547', 'LudRules_tfidf_548', 'LudRules_tfidf_550', 'LudRules_tfidf_551', 'LudRules_tfidf_553', 'LudRules_tfidf_554', 'LudRules_tfidf_555', 'LudRules_tfidf_556', 'LudRules_tfidf_558', 'LudRules_tfidf_559', 'LudRules_tfidf_563', 'LudRules_tfidf_564', 'LudRules_tfidf_565', 'LudRules_tfidf_567', 'LudRules_tfidf_568', 'LudRules_tfidf_569', 'LudRules_tfidf_570', 'LudRules_tfidf_571', 'LudRules_tfidf_573', 'LudRules_tfidf_574', 'LudRules_tfidf_575', 'LudRules_tfidf_576', 'LudRules_tfidf_577', 'LudRules_tfidf_579', 'LudRules_tfidf_585', 'LudRules_tfidf_586', 'LudRules_tfidf_587', 'LudRules_tfidf_588', 'LudRules_tfidf_589', 'LudRules_tfidf_590', 'LudRules_tfidf_593', 'LudRules_tfidf_597', 'LudRules_tfidf_598', 'LudRules_tfidf_599']
        
        df.drop(['Id',#Id没有什么实际的意义,就是行号
         #在train里面nunique=1的列
         'Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating', 'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules', 'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire', 'TwoPlayer', 'Multiplayer', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 'Container', 'Board', 'PrismShape', 'ParallelogramShape', 'RectanglePyramidalShape', 'TargetShape', 'BrickTiling', 'CelticTiling', 'QuadHexTiling', 'Hints', 'PlayableSites', 'Component', 'DiceD3', 'BiasedDice', 'Card', 'Domino', 'Rules', 'SituationalTurnKo', 'SituationalSuperko', 'InitialAmount', 'InitialPot', 'Play', 'BetDecision', 'BetDecisionFrequency', 'VoteDecisionFrequency', 'ChooseTrumpSuitDecision', 'ChooseTrumpSuitDecisionFrequency', 'LeapDecisionToFriend', 'LeapDecisionToFriendFrequency', 'HopDecisionEnemyToFriend', 'HopDecisionEnemyToFriendFrequency', 'HopDecisionFriendToFriend', 'FromToDecisionWithinBoard', 'FromToDecisionBetweenContainers', 'BetEffect', 'BetEffectFrequency', 'VoteEffectFrequency', 'SwapPlayersEffectFrequency', 'TakeControl', 'TakeControlFrequency', 'PassEffectFrequency', 'SetCost', 'SetCostFrequency', 'SetPhase', 'SetPhaseFrequency', 'SetTrumpSuit', 'SetTrumpSuitFrequency', 'StepEffectFrequency', 'SlideEffectFrequency', 'LeapEffectFrequency', 'HopEffectFrequency', 'FromToEffectFrequency', 'SwapPiecesEffect', 'SwapPiecesEffectFrequency', 'ShootEffect', 'ShootEffectFrequency', 'MaxCapture', 'OffDiagonalDirection', 'Information', 'HidePieceType', 'HidePieceOwner', 'HidePieceCount', 'HidePieceRotation', 'HidePieceValue', 'HidePieceState', 'InvisiblePiece', 'End', 'LineDrawFrequency', 'ConnectionDraw', 'ConnectionDrawFrequency', 'GroupLossFrequency', 'GroupDrawFrequency', 'LoopLossFrequency', 'LoopDraw', 'LoopDrawFrequency', 'PatternLoss', 'PatternLossFrequency', 'PatternDraw', 'PatternDrawFrequency', 'PathExtentEndFrequency', 'PathExtentWinFrequency', 'PathExtentLossFrequency', 'PathExtentDraw', 'PathExtentDrawFrequency', 'TerritoryLoss', 'TerritoryLossFrequency', 'TerritoryDraw', 'TerritoryDrawFrequency', 'CheckmateLoss', 'CheckmateLossFrequency', 'CheckmateDraw', 'CheckmateDrawFrequency', 'NoTargetPieceLoss', 'NoTargetPieceLossFrequency', 'NoTargetPieceDraw', 'NoTargetPieceDrawFrequency', 'NoOwnPiecesDraw', 'NoOwnPiecesDrawFrequency', 'FillLoss', 'FillLossFrequency', 'FillDraw', 'FillDrawFrequency', 'ScoringDrawFrequency', 'NoProgressWin', 'NoProgressWinFrequency', 'NoProgressLoss', 'NoProgressLossFrequency', 'SolvedEnd', 'Behaviour', 'StateRepetition', 'PositionalRepetition', 'SituationalRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation', 'Clarity', 'Narrowness', 'Variance', 'Decisiveness', 'DecisivenessMoves', 'DecisivenessThreshold', 'LeadChange', 'Stability', 'Drama', 'DramaAverage', 'DramaMedian', 'DramaMaximum', 'DramaMinimum', 'DramaVariance', 'DramaChangeAverage', 'DramaChangeSign', 'DramaChangeLineBestFit', 'DramaChangeNumTimes', 'DramaMaxIncrease', 'DramaMaxDecrease', 'MoveEvaluation', 'MoveEvaluationAverage', 'MoveEvaluationMedian', 'MoveEvaluationMaximum', 'MoveEvaluationMinimum', 'MoveEvaluationVariance', 'MoveEvaluationChangeAverage', 'MoveEvaluationChangeSign', 'MoveEvaluationChangeLineBestFit', 'MoveEvaluationChangeNumTimes', 'MoveEvaluationMaxIncrease', 'MoveEvaluationMaxDecrease', 'StateEvaluationDifference', 'StateEvaluationDifferenceAverage', 'StateEvaluationDifferenceMedian', 'StateEvaluationDifferenceMaximum', 'StateEvaluationDifferenceMinimum', 'StateEvaluationDifferenceVariance', 'StateEvaluationDifferenceChangeAverage', 'StateEvaluationDifferenceChangeSign', 'StateEvaluationDifferenceChangeLineBestFit', 'StateEvaluationDifferenceChangeNumTimes', 'StateEvaluationDifferenceMaxIncrease', 'StateEvaluationDifferenceMaxDecrease', 'BoardSitesOccupied', 'BoardSitesOccupiedMinimum', 'BranchingFactor', 'BranchingFactorMinimum', 'DecisionFactor', 'DecisionFactorMinimum', 'MoveDistance', 'MoveDistanceMinimum', 'PieceNumber', 'PieceNumberMinimum', 'ScoreDifference', 'ScoreDifferenceMinimum', 'ScoreDifferenceChangeNumTimes', 'Roots', 'Cosine', 'Sine', 'Tangent', 'Exponential', 'Logarithm', 'ExclusiveDisjunction', 'Float', 'HandComponent', 'SetHidden', 'SetInvisible', 'SetHiddenCount', 'SetHiddenRotation', 'SetHiddenState', 'SetHiddenValue', 'SetHiddenWhat', 'SetHiddenWho',
          #训练集里有但是测试集里没有的列
         'num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1',
         #object列
         'Behaviour', 'StateRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation', 'Clarity', 'Decisiveness', 'Drama', 'MoveEvaluation', 'StateEvaluationDifference', 'BoardSitesOccupied', 'BranchingFactor', 'DecisionFactor', 'MoveDistance', 'PieceNumber', 'ScoreDifference', 'EnglishRules', 'LudRules', 'selection1', 'selection2', 'exploration_const1', 'exploration_const2', 'playout1', 'playout2', 'score_bounds1', 'score_bounds2',
         #构造完特征不要了
         'agent1','agent2'
        ]+drop_cols,axis=1,inplace=True,errors='ignore')#对于测试集中没有的列可以直接忽略 
        df=self.reduce_mem_usage(df)
        print(f"feature_count:{len(df.columns)}")
        print("-"*30)
        return df
    
    def RMSE(self,y_true,y_pred):
        return np.sqrt(np.mean((y_true-y_pred)**2))
    
    def train_model(self):
        self.train = self.FE(self.train, mode='train')

        # LightGBM parameters
        lgb_params = {
            "boosting_type": "gbdt", "objective": "regression", "metric": "rmse",
            'random_state': 1212,
            "max_depth": 6,
            "learning_rate": 0.022,
            "n_estimators": 10_000,
            'min_child_samples': 6,
            'subsample': 0.75,
            "colsample_bytree": 0.8,
            "colsample_bynode": 0.8,
            "verbose": -1,
            "reg_alpha": 0.1,
            "reg_lambda": 10,
            "extra_trees": True,
            'num_leaves': 123,
            "max_bin": 225,
            'device': 'gpu', 'gpu_use_dp': True,
        }

        # XGBoost parameters
        #xgb_params = {
         #   "objective": "reg:squarederror",
          #  "learning_rate": 0.022,
           # "n_estimators": 10_000,
            #"max_depth": 6,
            #"colsample_bytree": 0.8,
            #"subsample": 0.75,
            #"random_state": 1212,
            #'tree_method': 'gpu_hist',  # Enable GPU support if available
            #'device': 'gpu_predictor'  # For inference on GPU
        #}

        # CatBoost parameters
        cat_params = {
            "iterations": 10_000,
            "depth": 6,
            "learning_rate": 0.022,
            "loss_function": 'RMSE',
            "eval_metric": 'RMSE',
            "random_seed": 1212,
            "verbose": 200,
            "task_type": "GPU",  # Set to "CPU" if not using GPU
        }

        # Initialize models
        model_lgb = lgb.LGBMRegressor(**lgb_params)
        #model_xgb = XGBRegressor(**xgb_params)
        model_cat = CatBoostRegressor(**cat_params)

        print("start training")
        X = self.train.drop([self.target, 'GameRulesetName'], axis=1)
        GameRulesetName = self.train['GameRulesetName']
        y = self.train[self.target]
        oof_preds_lgb = np.zeros(len(X))
        #oof_preds_xgb = np.zeros(len(X))
        oof_preds_cat = np.zeros(len(X))

        gkf = GroupKFold(n_splits=self.num_folds)

        for fold, (train_index, valid_index) in enumerate(gkf.split(X, y, GameRulesetName)):
            print(f"fold:{fold}")

            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

            # Train LightGBM
            model_lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                          callbacks=[log_evaluation(50), early_stopping(100)])
            oof_preds_lgb[valid_index] = model_lgb.predict(X_valid)
            self.models_lgb.append(model_lgb)

            # Train XGBoost
            #model_xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
             #             early_stopping_rounds=100, verbose=50)
            #oof_preds_xgb[valid_index] = model_xgb.predict(X_valid)
            #self.models_xgb.append(model_xgb)

            # Train CatBoost
            model_cat.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
                          early_stopping_rounds=100)
            oof_preds_cat[valid_index] = model_cat.predict(X_valid)
            self.models_cat.append(model_cat)

            del X_train, X_valid, y_train, y_valid
            gc.collect()

        # Calculate RMSE for each model
        print(f"LightGBM RMSE: {self.RMSE(y.values, oof_preds_lgb)}")
        #print(f"XGBoost RMSE: {self.RMSE(y.values, oof_preds_xgb)}")
        print(f"CatBoost RMSE: {self.RMSE(y.values, oof_preds_cat)}")

        # Optionally, average the OOF predictions for ensembling
        oof_preds_ensemble = (oof_preds_lgb + oof_preds_cat) / 2
        print(f"Ensemble RMSE: {self.RMSE(y.values, oof_preds_ensemble)}")

    def infer_model(self, test):
        test = self.FE(test, mode='test')
        test.drop(['GameRulesetName'], axis=1, inplace=True)

        if len(self.models_lgb) and len(self.models_cat):
            # Get predictions from all models and average them for the ensemble
            preds_lgb = np.mean([model.predict(test) for model in self.models_lgb], axis=0)
            #preds_xgb = np.mean([model.predict(test) for model in self.models_xgb], axis=0)
            preds_cat = np.mean([model.predict(test) for model in self.models_cat], axis=0)
            
            # my version
            # Return the ensemble predictions (average of all models) # my version
            #return (preds_lgb + preds_xgb + preds_cat) / 3
            
            #Taken from @simmy's updated verison
            #LightGBM RMSE: 0.42437550426403947
            lgbp = 0.42437550426403947
            #XGBoost RMSE: 0.43211711670126346
            #xgbp = 0.43211711670126346
            #CatBoost RMSE: 0.4220611799069957   
            cbp = 0.4220611799069957  
            return (preds_lgb*1 + preds_cat*2) / 3
            #return (preds_lgb*lgbp + preds_xgb*xgbp + preds_cat*cbp) / (lgbp+xgbp+cbp)

            # Return the ensemble predictions (average of all models)
            #return (preds_lgb + preds_xgb + preds_cat) / 3
        else:
            return np.array([0.04439766262399326] * len(test))  # Default predictions if no models exist

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">train and predict</h1></span>

#### The ultimate goal of this competition is to predict the effects of agents on unknown games, so we also need to divide them according to game types during cross validation. Therefore, we have chosen groupkfold here.

In [ ]:
import os
import kaggle_evaluation.mcts_inference_server#MCTS比赛专用的API

preprocessor=Preprocessor(num_folds=5,train=train)

# Initialize a counter to keep track of prediction calls
counter = 0

def predict(test, submission):
    global counter
    if counter == 0:
        preprocessor.train_model()  
    counter += 1
    return submission.with_columns(pl.Series('utility_agent1', preprocessor.infer_model(test.to_pandas())))


inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()

In [ ]:
print()